In [ ]:
from selenium import webdriver
import platform

#for windows
from selenium.webdriver.edge.service import Service as EdgeService
from webdriver_manager.microsoft import EdgeChromiumDriverManager
#for mac
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

from collections import defaultdict

import time
from dotenv import load_dotenv
import os
load_dotenv()

In [ ]:
driver = None
def init():
    driver = None
    if(platform.system() == 'Windows'):
        driver = webdriver.Edge(service=EdgeService(EdgeChromiumDriverManager().install()))
    elif(platform.system() == 'Linux' or platform.system() == 'Darwin'):
        driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

    driver.get('https://www.linkedin.com/uas/login')
    
    username = os.getenv('USER_NAME')
    password = os.getenv('PASSWORD')
    elementID = driver.find_element(By.ID, 'username')
    elementID.send_keys(username)

    elementID = driver.find_element(By.ID,'password')
    elementID.send_keys(password)

    elementID.submit()
    return driver

if __name__ == '__main__':
    driver = init()

In [ ]:
# link = 'https://www.linkedin.com/in/christinaspringer/'
# link = 'https://www.linkedin.com/in/ajit-sakri/'
# link = 'https://www.linkedin.com/in/mbotticelli/'
link = "https://www.linkedin.com/in/christopherkmay/"
# link = 'https://www.linkedin.com/in/brianlangel/'
# link = 'https://www.linkedin.com/in/terekpeterson/'
# link = 'https://www.linkedin.com/in/sam-hassani-75b91431/'
# link = "https://www.linkedin.com/in/bablu-mahato-ab38741b2/"
# link = "https://www.linkedin.com/in/padmapriya-sridhar-629602208"
# link = "https://www.linkedin.com/in/rakshita-joshi-53063217a/"
# link = "https://www.linkedin.com/in/rahul-kumar-b177591b2/"
# link = "https://www.linkedin.com/in/rahul-choudhary-18407b1a0/"
# link = "https://www.linkedin.com/in/gagan-gupta-8046a91a8/"
# link = "https://www.linkedin.com/in/rohit-kumar-9b60aa20a/"
detail = defaultdict(lambda : "")
# driver.get(link)

In [ ]:
def scroll(driver):
    SCROLL_PAUSE_TIME = 2
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    for i in range(2):
    # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    

def add_link(link,option):
    return (link + ('' if(link[-1] == '/') else '/') + option)



First i will extract 
 description headline location name

In [ ]:

def scrap_general(driver,link):
    general_info =  defaultdict(lambda: "")
    general_info["description"] = ""
    general_info["headline"] = ""
    general_info["location"] = ""
    general_info["fullName"] = ""

    driver.get(link)
    scroll(driver)
    descriptionxPath = "/html/body/div[5]/div[3]/div/div/div/div[2]/div/div/main/section[2]/div[3]/div/div/div/span[1]"
    headlinexPath = "/html/body/div[5]/div[3]/div/div/div/div[2]/div/div/main/section[1]/div[2]/div[2]/div[1]/div[2]"
    locationxPath = "/html/body/div[5]/div[3]/div/div/div/div[2]/div/div/main/section[1]/div[2]/div[2]/div[2]/span[1]"
    fullNamexPath = "/html/body/div[5]/div[3]/div/div/div/div[2]/div/div/main/section[1]/div[2]/div[2]/div[1]/div[1]/h1"
    
    try:
        general_info["description"] = driver.find_element(By.XPATH,descriptionxPath).text.split('\n')[0]
    except NoSuchElementException:
        print(link+ " unable to find description")
    try:
        general_info["headline"] = driver.find_element(By.XPATH,headlinexPath).text
    except NoSuchElementException:
        print(link+" unable to find headline")
    try:
        general_info["location"] = driver.find_element(By.XPATH,locationxPath).text
    except NoSuchElementException:
        print(link+" unable to find location")
    try:
        general_info["fullName"] = driver.find_element(By.XPATH,fullNamexPath).text
        name_split = general_info["fullName"].split(' ')
        general_info["firstName"] = name_split[0]
        if(len(name_split) > 1):
            general_info["lastName"] = name_split[1]
    except NoSuchElementException:
        print(link+" unable to find fullName")
    return general_info

if __name__ == "__main__":
    detail["general"] = scrap_general(driver,link)
    print(detail['general'])
    
    

In [ ]:
def scrap_email_website(driver,link):
    email_website = defaultdict(lambda: "")
    email_website["email"] = ""
    email_website["website"] = ""

    contact_info_page_link = add_link(link,"overlay/contact-info/")
    driver.get(contact_info_page_link)

    try:
        email_website["email"] = driver.find_element(By.CSS_SELECTOR,".ci-email>div>a").text
    except NoSuchElementException:
        print(link+" unable to find email")
    # website_xpath = 
    # website_xpath = "/html/body/div[3]/div/div/div[2]/section/div/section[2]/ul/li/a"
    try:
        email_website["website"] = driver.find_element(By.CSS_SELECTOR,".ci-websites>ul>li>a").get_attribute("href")
    except NoSuchElementException:
        print(link+" unable to find website")
    return email_website

if __name__ == "__main__":
    detail["email_website"] = scrap_email_website(driver,link)
    print(detail['email_website'])


In [ ]:
# location code i have to improve


def scrap_experience(driver,link):
    page_link_experience = add_link(link,'details/experience')
    driver.get(page_link_experience)
    # driver.implicitly_wait(0.5)
    # scroll(driver)

    # allExperiencexPath = "/html/body/div[5]/div[3]/div/div/div/div[2]/div/div/main/section[4]/div[3]/ul"
    experience_blocks_xpath = '//*[@id="main"]/section/div[2]/div/div[1]/ul/li'
    # experience_element = driver.find_element(By.XPATH,allExperiencexPath).get_attribute('innerHTML')
    # experience_element = driver.find_element(By.XPATH,allExperiencexPath).find_elements(By.XPATH,"li")

    # experience = [exp.text for exp in experience_element]

    # exp_description_simple_path = "div:nth-child(1) > div:nth-of-type(2) > div:nth-of-type(2) > ul:nth-child(1) > li:nth-child(1) > div:nth-child(1) > ul:nth-child(1) > li:nth-child(1) >div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > span:nth-of-type(1)"
    exp_description_simple_xpath = "div/div[2]/div[2]"
    exp_description_multi_xpath = "div/div[2]/div[2]"
    # exp_head_path = "div:nth-child(1) > div:nth-of-type(2) > div:nth-of-type(2) > ul:nth-child(1) > li:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) div:nth-child(1) span:nth-child(1) span:nth-child(1)"
    exp_head_path = "div/div[2]/div"
    company_url_path = "div/div[2]/div/a"
    exp_multi_body_path = "div/div[2]/div[2]/ul/li/div/div/div/ul/li"
    # print(len(experience_element))

    def job_start_end_duration(head_dict,st_end_durn):
        head_dict['job_date_range'] = ""
        head_dict["duration"] = ""
        head_dict["job_start"] = ""
        head_dict["job-end"] = ""
        if(st_end_durn.find('·') != -1):
            st_end_durn = st_end_durn.split(" · ")
            # print(st_end_durn)
            st_end = st_end_durn[0]
            head_dict['job_date_range'] = st_end
            head_dict["duration"] = st_end_durn[1]
            if(st_end.find('-') != -1):
                st_end = st_end.split(" - ")
                head_dict["job_start"] = st_end[0]
                head_dict["job-end"] = st_end[1]
            else:
                head_dict["job_start"] = st_end 
        else:
            # print("here")
            head_dict["duration"] = st_end_durn

    def scrap_head(exp):
        head_elem = None
        try:
            head_elem = exp.find_element(By.XPATH,exp_head_path)
            # head_elem = [head_elem[i] for i in range(0,len(head_elem),2)]
            # print(head_elem.text)
        except NoSuchElementException:
            # print(exp.text)
            pass
        
        return head_elem

    def scrap_head_detail_multi(head):
        head_dict = defaultdict(lambda: "")
        head_dict["company"] = ""
        try:
            head_dict["company"] = head.find_element(By.CSS_SELECTOR,".t-bold>span:nth-child(1)").text
        except NoSuchElementException:
            pass
        try:
            st_end_durn = head.find_element(By.CSS_SELECTOR,".t-black--light:nth-last-child(2)>span:nth-child(1)").text
            job_start_end_duration(head_dict,st_end_durn)
        except NoSuchElementException:
            pass
        return head_dict

    def scrap_head_detail_simple(head):
        head_dict = defaultdict(lambda: "")
        head_dict["job"] = ""
        head_dict["place"] = ""
        try:
            head_dict["job"] = head.find_element(By.CSS_SELECTOR,".t-bold>span:nth-child(1)").text
        except NoSuchElementException:
            pass
        try:
            head_dict["place"] = head.find_element(By.CSS_SELECTOR,".t-normal:not(.t-black--light)>span:nth-child(1)").text
        except NoSuchElementException:
            pass
        is_operationFailed = False
        try:
            st_end_durn = head.find_element(By.CSS_SELECTOR,".t-black--light:nth-last-child(2)>span:nth-child(1)").text
            job_start_end_duration(head_dict,st_end_durn)
        except:
            is_operationFailed = True
            pass
        try:
            if(is_operationFailed):
                st_end_durn = head.find_element(By.CSS_SELECTOR,".t-black--light:nth-last-child(1)>span:nth-child(1)").text
                job_start_end_duration(head_dict,st_end_durn)
            else:
                head_dict["location"] = head.find_element(By.CSS_SELECTOR,".t-black--light:nth-last-child(1)>span:nth-child(1)").text
        except:
            pass
        return head_dict

    def scrap_multi_body(exp):
        multi_body = []
        try:
            multi_body = exp.find_elements(By.XPATH,exp_multi_body_path)
        except:
            pass
        return multi_body

    def scrap_subbody_head(head):
        head_dict = {}
        try:
            head_dict["job"] = head.find_element(By.CSS_SELECTOR,".t-bold>span:nth-child(1)").text
        except:
            pass
        try:
            head_dict["type"] = head.find_element(By.CSS_SELECTOR,".t-normal:not(.t-black--light)>span:nth-child(1)").text
        except:
            pass
        is_operationFailed = False
        try:
            st_end_durn = head.find_element(By.CSS_SELECTOR,".t-black--light:nth-last-child(2)>span:nth-child(1)").text

            job_start_end_duration(head_dict,st_end_durn) 
        except:
            is_operationFailed = True
            pass
        try:
            if(is_operationFailed):
                st_end_durn = head.find_element(By.CSS_SELECTOR,".t-black--light:nth-last-child(1)>span:nth-child(1)").text
                job_start_end_duration(head_dict,st_end_durn)
            else:
                head_dict["location"] = head.find_element(By.CSS_SELECTOR,".t-black--light:nth-last-child(1)>span:nth-child(1)").text
        except:
            pass
        return head_dict

    def scrap_description_simple(exp):
        simple_des = ""
        try:
            simple_des = exp.find_element(By.XPATH,exp_description_simple_xpath).text
            # print(simple_des)
        except:
            pass
        return simple_des

    def scrap_description_multi(exp_local):
        multi_des = ""
        try:
            multi_des = exp_local.find_element(By.XPATH,exp_description_multi_xpath).text
            # print(multi_des)
        except:
            pass
        return multi_des

    def scrap_url(block):
        url = ""
        try:
            url = block.find_element(By.XPATH,company_url_path).get_attribute("href")
        except:
            pass
        return url

    allexperience = []
    all_experience_block = []
    try:
        all_experience_block = WebDriverWait(driver, timeout=3).until(lambda d: d.find_elements(By.XPATH,experience_blocks_xpath))
    except TimeoutException:
        print(link+" timeout occur in experience section consider increasing timeout")
        return allexperience
    # all_experience_block = driver.find_elements(By.XPATH,experience_blocks_xpath)
    for exp in all_experience_block:
        try:
            simple_des = True
            # print(exp.text,'\n\n')
            head = scrap_head(exp)
            # print(head.text)
            url = scrap_url(exp)
            # if(head):
            #     print (head)
                
            #check if exp is simple experience block or it is multiexperience block
            multiple_exp_blocks = scrap_multi_body(exp)
            # simple_desc = scrap_description_simple(exp)
            # print(len(multiple_exp_blocks))
            if(len(multiple_exp_blocks) > 1):
                #exp is multibody block
                head_detail = scrap_head_detail_multi(head)
                for exp_local in multiple_exp_blocks:
                    # print(exp_local.text)
                    head_sub_body = None
                    try:
                        head_sub_body = scrap_subbody_head(scrap_head(exp_local))
                    except:
                        # print(head_sub_body)
                        pass
                    if(head_sub_body):
                        # print(head_sub_body)
                        head_sub_body = [head_detail, head_sub_body]
                        # print(head_sub_body)

                    description = scrap_description_multi(exp_local)
                    allexperience.append([head_sub_body,description,url])
            else:
                #exp is simple body block
                # print(head)
                head_detail = scrap_head_detail_simple(head)
                description = scrap_description_simple(exp)
                allexperience.append([head_detail,description,url])
        except:
            pass
    
    return allexperience
    
if __name__ == "__main__":
    
    detail["experience"] = scrap_experience(driver,link)
    print(detail['experience'])
    



In [ ]:
def scrap_education(driver,link):
    # allEducationxPath = "/html/body/div[5]/div[3]/div/div/div/div[2]/div/div/main/section[5]/div[3]/ul/li"
    page_link_education = add_link(link,'details/education')
    # driver.implicitly_wait(0.5)
    # scroll(driver)
    driver.get(page_link_education)
    education_blocks_xpath = '//*[@id="main"]/section/div[2]/div/div[1]/ul/li'
    head_block_xpath = "div/div[2]/div[1]"


    def scrap_edu_block_head(edu):
        edu_head = None
        try:
            edu_head = edu.find_element(By.XPATH,head_block_xpath)
        except:
            pass
        return edu_head

    def scrap_head_detail(head):
        head_dict = {}
        try:
            head_dict["college"] = head.find_element(By.CSS_SELECTOR,".t-bold>span:nth-child(1)").text
        except:
            pass
        try:
            head_dict["degree"] = head.find_element(By.CSS_SELECTOR,".t-normal:not(.t-black--light)>span:nth-child(1)").text
        except:
            pass
        try:
            head_dict["timeline"] = head.find_element(By.CSS_SELECTOR,".t-black--light>span:nth-child(1)").text
        except:
            pass
        return head_dict

    allEducation = []
    all_EducationBlock = []
    try:
        all_EducationBlock = WebDriverWait(driver, timeout=3).until(lambda d: d.find_elements(By.XPATH,education_blocks_xpath))
    except TimeoutException:
        print(link+ " timeout occur in scrapping education consider increasing it")
        return allEducation
    # all_EducationBlock = driver.find_elements(By.XPATH,education_blocks_xpath)
    for edu in all_EducationBlock:
        try:
            head = scrap_edu_block_head(edu)
            if(head):
                head = scrap_head_detail(head)
                # head = head.text.split('\n')[::2]
                # print(head)
                allEducation.append(head)
        except:
            pass

    # for edu in allEducation:
    #     print(edu)
    return allEducation
if __name__ == "__main__":
    detail["education"] = scrap_education(driver,link)
    print(detail['education'])

In [ ]:

#scrap license
# here license and certificate is named only certificate

def scrap_certificate(driver,link):
    certificate_page_link = add_link(link,"details/certifications/" )

    driver.get(certificate_page_link)
    # driver.implicitly_wait(0.5)
    # scroll(driver)
    certificate_blocks_xpath = '//*[@id="main"]/section/div[2]/div/div[1]/ul/li'

    # print(certificate_blocks)


    def scrap_head_certificate(certificate):
        head_path = "div/div[2]/div[1]"
        head_elem = None
        try:
            head_elem = certificate.find_element(By.XPATH,head_path)
        except:
            pass
        return head_elem

    def scrap_certificate_credential(certificate):
        credential_path = "div/div[2]/div[2]/ul/li/div/a"
        credential_link = "" 
        try:
            credential_link = certificate.find_element(By.XPATH,credential_path).get_attribute("href")
        except:
            pass
        return credential_link

    def scrap_head_detail(head):
        head_dict = {}
        try:
            head_dict["name"] = head.find_element(By.CSS_SELECTOR,".t-bold>span:nth-child(1)").text
        except:
            pass
        try:
            head_dict["source"] = head.find_element(By.CSS_SELECTOR,".t-normal:not(.t-black--light)>span:nth-child(1)").text
        except:
            pass
        return head_dict

    allcertificate = []
    certificate_blocks = []
    try:
        certificate_blocks = WebDriverWait(driver, timeout=3).until(lambda d: d.find_elements(By.XPATH,certificate_blocks_xpath))
    except TimeoutException:
        print(link+ "timeout occur in certificate")
        return allcertificate
    # certificate_blocks = driver.find_elements(By.XPATH,certificate_blocks_xpath)
    for certificate in certificate_blocks:
        try:
            head_block = scrap_head_certificate(certificate)
            head = scrap_head_detail(head_block)
            certificate_url = scrap_certificate_credential(certificate)
            allcertificate.append([head,certificate_url])
        except:
            pass
    
    return allcertificate

if __name__ == "__main__":
    detail["certificate"] = scrap_certificate(driver,link)
    print(detail['certificate'])

In [ ]:
def scrap_skill(driver,link):
    skill_page_link = add_link(link,"details/skills/")
    driver.get(skill_page_link)
    driver.implicitly_wait(0.5)

    skill_blocks_xpath = '//*[@id="main"]/section/div[2]/div[2]/div/div/div[1]/ul/li'

    endorsement_count_xpath = 'div/div[2]/div[2]/ul/li[last()]'
    # print(skill_blocks)

    def scrap_skill_block(skill):
        skill_block = None    
        try:
            # skill_block = WebDriverWait(skill, timeout=3).until(lambda d: d.find_element(By.XPATH,"div/div[2]/div[1]/div/div/span/span[1]" ))
            skill_block = skill.find_element(By.XPATH,"div/div[2]/div[1]/div/div/span/span[1]")
        except:
            pass
        return skill_block


    def scrap_endorsement(skill):
        endorsement = 0
        try:
            # endorsement = WebDriverWait(skill, timeout=3).until(lambda d: d.find_element(By.XPATH,endorsement_count_xpath))
            endorsement = int(skill.find_element(By.XPATH,endorsement_count_xpath).text.split(' ')[0])
            # endorsement = endorsement.text.split(' ')[0]
            # print(endorsement)
        except:
            pass
        return endorsement


    all_skill = []
    skill_blocks = []
    # wait = WebDriverWait(driver, timeout=10, poll_frequency=1, ignored_exceptions=[ElementNotVisibleException, ElementNotSelectableException])
    try:
        skill_blocks = WebDriverWait(driver, timeout=3).until(lambda d: d.find_elements(By.XPATH,skill_blocks_xpath))
    except TimeoutException:
        print(link+" timeout occur in skill block")
        return all_skill
    # skill_blocks = driver.find_elements(By.XPATH,skill_blocks_xpath)
    for skill in skill_blocks:
        try:
            skill_name = scrap_skill_block(skill).text
            # print(skill_name)
            endorsement = scrap_endorsement(skill)
            all_skill.append([skill_name,endorsement])
        except:
            pass
    
    return all_skill

if __name__ == "__main__":
    detail["skill"] = scrap_skill(driver,link)
    print(detail['skill'])


In [ ]:

#this function will not work if i am currently in another tab
def scrap_activity(driver,link):
    activity_page_link = add_link(link,"recent-activity/")
    # print(link)
    # print(activity_page_link)
    
    activity_blocks_urns_xpath = '//*[@id="main"]/div/div[2]/div/div/div/div/div'

    all_activity_urls = []
    try:
        driver.get(activity_page_link)
        driver.implicitly_wait(0.5)
        scroll(driver)
        
    except:
        print("error in opening activity page")
        return all_activity_urls
    finally:
        all_activity_blocks = []
        try:
            all_activity_blocks = WebDriverWait(driver, timeout=3).until(lambda d:  d.find_elements(By.XPATH,activity_blocks_urns_xpath))
        except TimeoutException:
            print(link+" timeout occur in activity")
            return all_activity_urls
        except:
            print("error in fetching all activity")
        for activity in all_activity_blocks:
            try:
                url = "https://www.linkedin.com/feed/update/" + activity.get_attribute('data-urn')
                all_activity_urls.append(url)
            except:
                print("unable to fetch data-urn")
    return all_activity_urls

if __name__ == "__main__":
    detail["activity"] = scrap_activity(driver,link)
    print(detail['activity'])


In [ ]:
import csv
import json
# from os import exists
from dotenv import load_dotenv
import os
load_dotenv()

outcsvfile = os.getenv('OUT_FILE')

company_threashold = int(os.getenv('COMPANY_THREASHOLD'))
skill_threashold = int(os.getenv('SKILL_THREASHOLD' ))
education_threashold = int(os.getenv('EDUCATION_THREASHOLD' ))
certificate_threashold = int(os.getenv('CERTIFICATE_THREASHOLD' ))
activity_threashold = int(os.getenv('ACTIVITY_THREASHOLD' ))

general = ['linkedinProfile','email','description','headline','location','firstName','lastName','fullName','linkedinSalesNavigatorUrl']
company = [name+str(i) for i in range(1,company_threashold+1)  for name in ['company','companyUrl','jobTitle','jobDescription','jobLocation','jobDateRange'] ]
skill = ['skill'+str(i) for i in range(1,skill_threashold+1)]
education = [name+str(i)  for i in range(1,education_threashold+1) for name in ['school','schoolDegree','schoolDateRange']]
certificate = [name + str(i)  for i in range(1,certificate_threashold+1) for name in ['certificate','source']]
activity = ['Activity'+ str(i) for i in range(1,activity_threashold+1)]

fieldnames = [*general,*company,'website',*skill,*education,*certificate,*activity]

def writedef(dict,key1,key2=None):
    if(key2):
        if(key1 and key1 in dict):
            if(key2 in dict[key1]):
                return dict[key1][key2]
            else:
                return ''
        else:
            return ''
    elif(key1):
        if(key1 in dict):
            return dict[key1]
        else:
            return '' 
    else:
        return ''

def dumptocsv(detail):
    csvfile = None
    writer = None
    if os.path.exists(outcsvfile):
        csvfile = open(outcsvfile,'a',encoding="utf-8",newline='')
        header = csv.reader(csvfile)
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        pass
    else:
        csvfile = open(outcsvfile,'a',encoding="utf-8",newline='')
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

    dict = {}
    dict['linkedinProfile'] = detail['url']
    dict['email'] = writedef(detail,"email_website",'email')
    dict['website'] = writedef(detail,'email_website','website')
    dict['description'] = writedef(detail,'general','description')
    dict['headline'] = writedef(detail,'general','headline')
    dict['location'] = writedef(detail,'general','location')
    dict['firstName'] = writedef(detail,'general','firstName')
    dict['lastName'] = writedef(detail,'general','lastName')
    dict['fullName'] = writedef(detail,'general','fullName')

    i = 1
    while i <= company_threashold and i <= len(detail['experience']):
        exp = detail['experience'][i-1]
        if type(exp[0]) is list:
            dict['company'+str(i)] = writedef(exp[0][0],'company')
            dict['companyUrl'+str(i)] = exp[2]
            dict['jobTitle'+str(i)] = writedef(exp[0][1],'job')
            dict['jobDescription'+str(i)] =  exp[1]
            dict['jobLocation'+str(i)] = writedef(exp[0][1],'location')
            dict['jobDateRange'+str(i)] = writedef(exp[0][1],'job_date_range')
        else:
            dict['company'+str(i)] = writedef(exp[0],'place')
            dict['companyUrl'+str(i)] = exp[2]
            dict['jobTitle'+str(i)] = writedef(exp[0],'job')
            dict['jobDescription'+str(i)] =  exp[1]
            dict['jobLocation'+str(i)] = writedef(exp[0],'location')
            dict['jobDateRange'+str(i)] = writedef(exp[0],'job_date_range')
        i+=1
    
    i = 1
    while i <= skill_threashold and i <= len(detail['skill']):
        skill = detail['skill'][i-1]
        dict['skill'+str(i)] = skill[0]
        i+=1
    
    i = 1
    while i <= education_threashold and i <= len(detail['education']):
        edu = detail['education'][i-1]
        dict['school'+str(i)] = writedef(edu,'college')
        dict['schoolDegree'+str(i)] = writedef(edu,'degree')
        dict['schoolDateRange'+str(i)] = writedef(edu,'timeline')
        i+=1

    i = 1
    while i<= certificate_threashold and i <= len(detail['certificate']):
        cert = detail['certificate'][i-1]
        dict['certificate'+str(i)] = writedef(cert[0],'name')
        dict['source'+str(i)] = writedef(cert[0],'source')
        i+=1
    i = 1
    while i <= activity_threashold and i <= len(detail['activity']):
        dict['Activity'+str(i)] = detail['activity'][i-1]
        i+=1
    writer.writerow(dict)
    csvfile.close()

In [ ]:
if __name__ == '__main__':
    detail["url"] = link
    dumptocsv(detail)

In [ ]:
def trytoget(detail,field,func,driver,link):
    detail[field] = func(driver,link)
    

def scrap_all(driver,link):
    try:
        detail = {}
        detail['url'] = link
        detail['experience'] = []
        detail['skill'] = []
        detail['education'] = []
        detail['certificate'] = []
        detail['activity'] = []
        trytoget(detail,"general",scrap_general,driver,link)
        trytoget(detail,"email_website",scrap_email_website,driver,link)
        trytoget(detail,"experience",scrap_experience,driver,link)
        trytoget(detail,"education",scrap_education,driver,link)
        trytoget(detail,"certificate",scrap_certificate,driver,link)
        trytoget(detail,"skill",scrap_skill,driver,link)
        trytoget(detail,"activity",scrap_activity,driver,link)
        dumptocsv(detail)
    except:
        pass
    